In [ ]:
from pathlib import Path

import pandas as pd

pd.set_option('display.max_columns', 100)

In [ ]:
REPO_ROOT = Path.cwd().parent.parent
runs_path = REPO_ROOT / 'outputs/runs-sem-seg.csv'
assert runs_path.exists(), f'Runs not found: {runs_path}'

In [ ]:
df = pd.read_csv(runs_path)
df['Start Time'] = pd.to_datetime(df['Start Time'])
df.head()

In [ ]:
df_sel = df[
    (df['Start Time'] >= pd.to_datetime('2025-02-28-08', format='%Y-%m-%d-%H'))
    & (df['Start Time'] < pd.to_datetime('2025-03-01'))
].copy()
df_sel

In [ ]:
# Fill missing decoder/encoder parameters:
# -unet & enet-b0 (4007548, 2243776)
# -unet++ & enet-b0 (4007548, 2561888)

df_sel['encoder_params'] = df_sel['encoder_params'].fillna(4007548)
df_sel.loc[
    df_sel['decoder_params'].isna() & (df_sel['decoder'] == 'unet'), 'decoder_params'
] = 2243776
df_sel.loc[
    df_sel['decoder_params'].isna() & (df_sel['decoder'] == 'unetplusplus'),
    'decoder_params',
] = 2561888

In [ ]:
PARAMS_COLUMNS = ['encoder_params', 'decoder_params']
METRICS_COLUMS = [
    'test_f1_score',
    'test_jaccard_index',
    'test_f1_score',
    'test_jaccard_index',
]
MODEL_COLUMNS = ['encoder', 'decoder']

In [ ]:
df_table = df_sel[MODEL_COLUMNS + PARAMS_COLUMNS + METRICS_COLUMS].copy()
df_table[PARAMS_COLUMNS] = df_table[PARAMS_COLUMNS].div(1e6).round(1)
df_table['decoder'] = df_table['decoder'].replace(
    {'unet': 'U-Net', 'unetplusplus': 'UNet++'}
)
df_table['encoder'] = (
    df_table['encoder']
    .str.replace('efficientnet-b', 'EfficientNet-B')
    .replace('resnet', 'ResNet')
)
df_table['encoder'] = df_table['encoder'].str.replace('resnet', 'ResNet')
df_table[PARAMS_COLUMNS] = df_table[PARAMS_COLUMNS].astype(str)
df_table['encoder'] += ' (' + df_table['encoder_params'] + 'M)'
df_table['decoder'] += ' (' + df_table['decoder_params'] + 'M)'

df_export = df_table.drop(columns=PARAMS_COLUMNS).sort_values(
    by=['encoder', 'decoder'], ascending=True
)
df_export.columns = pd.MultiIndex.from_tuples(
    [
        (arch := 'Architecture', 'Decoder (\\#params)'),
        (arch, 'Encoder (\\#params)'),
        (test_set := 'Test set', 'F1 score'),
        (test_set, 'IoU'),
        (val_set := 'Validation set', 'F1 score'),
        (val_set, 'IoU'),
    ]
)
df_export

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_latex.html
df_export.to_latex(
    'table-sem-seg-results.tex',
    index=False,
    float_format='%.4f',
    multicolumn_format='c',
    caption='Best performing models.',
    label='tab:sem-seg-results',
    position='htbp',
)